In [10]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

## **Task**

We want to train an MLP to learn the construction of names. To achieve this we want to maximise the likelihood of the $N$ names observed in the dataset:

$$
\max_{\theta} \sum_{i=1}^{N} \hat{p}(\mathbf{x}_i) = \max_{\theta} \sum_{i=1}^{N} \prod_{j=1}^{M_i} \hat{p}(x_j | x_{j-1}, x_{j-2}, x_{j-3}; \theta)
$$

In particular, we choose to break down the probability of a name into the product of the conditional probabilities of each character given a three-letter context window (we use '.' as the special token to pad the start and end of a name). For example, the name "John" would give us the following data points:

* ... $\rightarrow$ J
* ..J $\rightarrow$ o
* .Jo $\rightarrow$ h
* Joh $\rightarrow$ n
* ohn $\rightarrow$ .

We begin by constructing a training set of such data points from all the names in the dataset. We then use the negative log-likelihood of the data as the loss function to be minimized. See `mm_intro.ipynb` for more details.

# **Dataset**

In [2]:
# Load data
words = open('names.txt', 'r').read().splitlines()
print(words[:10])

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn']


In [3]:
# Create a dictionary that maps characters to integers and vice versa
char2idx = {c: i+1 for i, c in enumerate('abcdefghijklmnopqrstuvwxyz')}
char2idx['.'] = 0 # special character for marking start and end of a word
idx2char = {i: c for c, i in char2idx.items()}

In [4]:
# Form training pairs of context and target characters
block_size = 3 # context size for next character prediction
X, Y = [], []

for word in words[:5]:
    w2idx = [0] * block_size + [char2idx[c] for c in word] + [0]
    for i in range(len(w2idx) - block_size):
        X.append(w2idx[i:i+block_size])
        Y.append(w2idx[i+block_size])

X = torch.tensor(X)
Y = torch.tensor(Y)

print('X:', X.shape, X.dtype, '\nY:', Y.shape, Y.dtype)
# print first 5 samples of X and Y
# for i in range(len(Y)):
#     print(''.join([idx2char[idx.item()] for idx in X[i]]), '->', idx2char[Y[i].item()])

X: torch.Size([32, 3]) torch.int64 
Y: torch.Size([32]) torch.int64


# **MLP**

We will use a simple MLP with an embedding layer for the input (shared across all characters), one hidden layer, and a softmax output layer. The approach is similar to the one used in the paper [Bengio et al. 2003](https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf).

1. **Embedding Layer**: As done in `mm_intro.ipynb`, characters are represented as one-hot vectors and passed through an embedding layer to obtain a dense representation. Regarding the implementation, we observe that a matrix multiplication of one-hot vectors with an embedding matrix is equivalent to a lookup in the embedding matrix. This can be implemented using simple indexing operations in PyTorch which is a lot more efficient than matrix multiplication: `emb = C[X]`.
2. **Hidden Layer**: The hidden layer is a fully connected layer with weights and biases. The input to the hidden layer is the concatenation of the embeddings of the three characters in the context window. To perform this concatenation, we use the `view` function in PyTorch which is extremely efficient as it doesn't directly manipulate the data in memory but simply changes its interpretation. 
3. **Output Layer**: The output layer is a fully connected layer with weights and biases. The output of the hidden layer is passed through this layer to obtain the logits for each character in the vocabulary. 
4. **Loss**: Instead of manually implementing the softmax and cross-entropy loss to compute the negative log-likelihood from the logits as we did in `mm_intro.ipynb`, we use PyTorch's `torch.nn.functional.cross_entropy` function which combines the two operations into one and is numerically not only more stable but also more efficient in the forward and backward passes. This is because the softmax and cross-entropy operations are fused into a single kernel and computed together. For more info, refer to the [YoutTube video](https://youtu.be/TCH_1BHY58I?t=1968) associated with this tutorial.

Note that when implementing models in PyTorch, it's always important to double-check the shapes of the tensors at each step using a small batch of data to make sure that the operations are being performed as intended.

In [17]:
g = torch.Generator().manual_seed(2147483647)

# Embedding layer
C = torch.randn((27, 2), requires_grad=True, generator=g) # shape: (vocab_size, embedding_dim)
# Hidden layer
W1 = torch.randn((6, 100), requires_grad=True, generator=g) # shape: (block_size * embedding_dim, hidden_dim)
b1 = torch.randn((100), requires_grad=True, generator=g) # shape: (hidden_dim)
# Output layer
W2 = torch.randn((100, 27), requires_grad=True, generator=g) # shape: (hidden_dim, vocab_size)
b2 = torch.randn((27), requires_grad=True, generator=g) # shape: (vocab_size)

parameters = [C, W1, b1, W2, b2]
print('Number of parameters:', sum([p.numel() for p in parameters]))

Number of parameters: 3481


## Training

Once you have built your model it's good practice to check that it's working as expected by overfitting it on a small batch of data. This means that the model should be able to memorize the data and achieve a loss of 0. 

In our case, we overfit the model to the first five entries of our `words` dataset by rerunning the training set construction code above with `words[:5]`. We then train the model for 1000 epochs and observe that the loss does indeed decrease significantly (to $0.2535$). The reason, however, why we are not able to achieve a loss of $0$, despite having a relatively large model of $3481$ parameters for only five words (32 samples), is because in some cases there is more than one possible next character given the context window. For example, in the 32 samples, the context window "..." appears multiple times and is followed by different characters. Hence, there is some inherent ambiguity in the data which prevents the model from achieving a loss of $0$.

To compare the model's predictions with our labels `Y`, we can use the `torch.argmax` function along the first dimension of the `logits` to obtain the index of the character with the highest probability and compare the two.

In [22]:
# Training loop
for _ in range(1000):
    # forward pass
    emb = C[X] # shape: (#samples, 3, 2) 
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # shape: (#samples, hidden_dim) 
    logits = h @ W2 + b2 # shape: (#samples, vocab_size)
    loss = F.cross_entropy(logits, Y) # shape: (1)

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # update parameters
    for p in parameters:
        p.data -= 0.1 * p.grad

print('Loss:', loss.item())

Loss: 0.25359871983528137
